# Solve combinatoric metaproblem

Posted as https://twitter.com/Avi_Maths/status/1674605793109331968 :

Consider the following statements:

1. A is older than B
2. C and D are of the same age
3. E is the youngest
4. F is younger than D
5. F is older than A

How many of the statements above are required to determine the oldest person or persons?

a. Only two  
b. Only three  
c. Only four  
d. All five  

***
First we need to define all canonical orderings:

In [13]:
from itertools import combinations as comb
from itertools import permutations as perm
from itertools import product as prod

# Given six items A B C D E F
# generate all unique orderings where some items may be equal,
# eg  C > (D=E) > F > (A=B)
# then encode each as the ordering-indices,
# eg (3, 3, 0, 1, 1, 2)

all_orderings = set()
# working array - reuse it to avoid memory churn
ordering = [0] * 6

# Generate the item in each position
for u,v,w,x,y,z in perm(range(6)):
    # Generate comparatives,
    #   0 = next item is equal to current item
    #   1 = next item is less than current item
    # N.B. this will produce some duplicates, eg A=B and B=A,
    #   but it will be taken care of by the position-encoding
    for uv,vw,wx,xy,yz in prod([0, 1], repeat=5):
        pos = 0
        ordering[u] = pos
        pos += uv
        ordering[v] = pos
        pos += vw
        ordering[w] = pos
        pos += wx
        ordering[x] = pos
        pos += xy
        ordering[y] = pos
        pos += yz
        ordering[z] = pos
        # now save it
        all_orderings.add(tuple(ordering))

This generates 4,683 unique orderings such as

    (2, 0, 3, 1, 4, 1)  ->  B > (D=F) > A > C > E  
    (1, 4, 0, 0, 2, 3)  ->  (C=D) > A > E > F > B  
    (0, 0, 2, 1, 2, 3)  ->  (A=B) > D > (C=E) > F  

***
Second, we define our rule-set.

In [14]:
def rule_1(order):
    # A is older than B
    return order[0] < order[1]

def rule_2(order):
    # C is the same age as D
    return order[2] == order[3]

def rule_3(order):
    # E is the youngest
    e_group = order[4]
    return all(i == 4 or ch_group < e_group for i, ch_group in enumerate(order))

def rule_4(order):
    # D is older than F
    return order[3] < order[5]

def rule_5(order):
    # F is older than A
    return order[5] < order[0]

***
For efficiency, we will only run each rule against each ordering once and cache the results.

We will also get and cache the oldest person/persons for each ordering.

In [15]:
def get_oldest(order):
    return "".join(
        ch
        for ch, grp in zip("ABCDEF", order)
        if grp == 0
    )

results = [
    (order, rule_1(order), rule_2(order), rule_3(order), rule_4(order), rule_5(order), get_oldest(order))
    for order in all_orderings
]

This generates 4,683 results like

    ((2, 0, 3, 1, 4, 1), False, False, True, False, True, 'B')  
    ((1, 4, 0, 0, 2, 3), True, True, False, True, False, 'CD')  
    ((0, 0, 2, 1, 2, 3), False, False, False, True, False, 'AB')  

***
Now let's find out how many rules are actually needed:

In [16]:
# How many rules to try?
for num_rules in range(1, 6):    # 1 .. 5
    # Pick that many rules
    for which_rules in comb(range(1, 6), num_rules):
        # Collect all orderings that pass that rule-set
        orders = []
        oldest = set()
        failed = False
        for order in results:
            if all(order[rule] for rule in which_rules):
                orders.append(order)
                oldest.add(order[6])
                if len(oldest) > 1:
                    # not a unique result!!
                    failed = True
                    break
        # did we find a solution?
        if not failed:
            print("Successful rule-set: ", which_rules, "finds", len(orders), "possible orderings where oldest is", oldest)


Successful rule-set:  (1, 2, 3, 4, 5) finds 1 possible orderings where oldest is {'CD'}
